

https://www.kaggle.com/competitions/used-cars-price-prediction-22ds соревнование

https://dzen.ru/a/Y_W7RVk7mG5MUBlh статья по ML

https://github.com/menadzhiev/car_price_predictor/blob/main/%D0%A1%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5%20%D0%BD%D0%BE%D0%B2%D1%8B%D1%85%20%D0%BF%D1%80%D0%B8%D0%B7%D0%BD%D0%B0%D0%BA%D0%BE%D0%B2.ipynb

https://www.dmitrymakarov.ru/

Описание полей данных



1. Date: Год выпуска автомобиля.
2. Make: Марка автомобиля.
3. Model: издание автомобиля определенной марки.
4. Trim: Уровни отделки салона автомобиля — это просто разные версии модели.
5. Body: Тип кузова транспортного средства относится к форме и модели конкретной марки автомобиля.
6. Transmission: механизм, который передает мощность от двигателя к колесам.
7. VIN: идентификационный номер транспортного средства.
8. State: состояние, в котором автомобиль выставлен на аукцион.
9. Condition: Состояние автомобилей на момент аукциона.
10. Odometer: расстояние, пройденное автомобилем с момента выпуска.
11. Color: Цвет кузова автомобиля.
12. Interior: Цвет салона автомобиля.
13. Seller: Продавец автомобиля, автосалоны.
14. mmr: Рекорд рынка Manhiem, рыночная оценочная цена автомобилей.
15. sellingprice: цена, по которой автомобиль был продан на аукционе.
16. saledate: Дата продажи автомобиля.




План работы
1. Загрузка данных и библиотек для работы
2. Обработка датасетов
3. Изучение данных
4. Постановка задачи для ML
5. Изучение лучших моделей обучения
6. Тестирование всех моделей на тестовой выборке
7. выбор наилучшей модели
8. Вывод


###1. Загрузка данных и библиотек для работы

In [ ]:
# Библиотеки используемые в работе
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler
)


from sklearn.model_selection import cross_val_score

from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


from sklearn.model_selection import train_test_split

In [ ]:
# загрузка датасетов
train_db = pd.read_csv('train.csv')
test_db = pd.read_csv('test.csv')


**Вывод:** Были импортированны базовые библиотеки и скачены основные датасеты для работы. Тренировочный и тестовый датасет отображаются без проблем.

###2. Обработка датасетов





In [ ]:
def base_analizez(data):
    ''' функция для изучения датасета'''

    print('Просмотр датасета:')
    display(data.head())

    print('\n')
    print('Общая информация об датасете:')
    print(data.info())

    print('\n')
    print('Проверка на наличие значений nan в датасете:')
    display(pd.DataFrame(round(
        data.isna().mean()*100,)).style.background_gradient('coolwarm'))

    # print('\n')
    # print('Изучение корреляции между данными')

    # Т.к. в датасете присутствует большое количество категориальных
    # переменных применим метрику Phik (φk) корреляции.

    # phik_overview = data.phik_matrix()

    # plot_correlation_matrix(phik_overview.values.round(2),
    #                     x_labels=phik_overview.columns,
    #                     y_labels=phik_overview.index,
    #                     vmin=0, vmax=1, color_map="Greens",
    #                     title=r"correlation $\phi_K$",
    #                     fontsize_factor=1,
    #                     figsize=(15, 8))
    # plt.tight_layout()

In [ ]:
# подробное изучения тренировойчного датасета
base_analizez(train_db)

In [ ]:
# изучение тренировочного датасета
base_analizez(test_db)

In [ ]:
%%time
# Заменим пропуски на самые популярные значения по модели ( в дальнейшем можно применить ML для этого)
def change_transmission(data):
    '''меняет пустые значения на самые распрастраненные'''

    new_transmission = {}

    for i in data['model'].unique():
        counted = []
        try:
            for x in data['transmission'].unique():
                if x == 'Nan':
                    pass
                counted.append([x, data.loc[(data['model'] == i) & (data['transmission'] == x), ['transmission']].count()])
        except:
            counted.append('Nan')
        try:
            if counted[0][1] >= counted[1][0]:
                new_transmission[i] = counted[0][0]
            else:
                new_transmission[i] = counted[1][0]
        except:
            new_transmission[i] = counted[0][0]

    for x in data['model'].unique():
        data.loc[(data['model'] == x) & (data['transmission'].isna()), 'transmission'] = new_transmission[x]

    return data

In [ ]:
train_db['make'] = train_db['make'].fillna('base')
train_db['model'] = train_db['model'].fillna('base1')
train_db['body'] = train_db['body'].fillna('base2')
train_db['trim'] = train_db['trim'].fillna('base3')
train_db['condition'] = train_db['condition'].fillna(2.5)
train_db['color'] = train_db['color'].fillna('base4')
train_db['odometer'] = train_db['odometer'].fillna(10000)
train_db['interior'] = train_db['color'].fillna('base5')

In [ ]:
test_db['make'] = test_db['make'].fillna('base')
test_db['model'] = test_db['model'].fillna('base1')
test_db['body'] = test_db['body'].fillna('base2')
test_db['trim'] = test_db['trim'].fillna('base3')
test_db['condition'] = test_db['condition'].fillna(2.5)
test_db['color'] = test_db['color'].fillna('base4')
test_db['odometer'] = test_db['odometer'].fillna(10000)
test_db['interior'] = test_db['color'].fillna('base5')

In [ ]:
from tqdm import tqdm
for i in tqdm(range(1)):
# заполняем пустые данные
    train_db = change_transmission(train_db)
    test_db = change_transmission(test_db)

In [ ]:
# удаляем из тренировочной модели оставшиеся не значительные пустые признаки
train_db = train_db.dropna(subset = ['make', 'model', 'body','trim', 'condition','color', 'odometer'])
# test_db = test_db.dropna(subset = ['make', 'model', 'body','trim', 'condition','color', 'odometer'])


In [ ]:
train_db.info()

In [ ]:
test_db.info()

In [ ]:
train_db.info()

In [ ]:
# выделим колонки с типомаи данных int и object для удобства
def int_or_object(data):
    columns_int = []
    columns_object = []
    for i in data.columns:
        if (data[i].dtype == np.float64 or data[i].dtype == np.int64):
            columns_int.append(i)
        else:
            columns_object.append(i)
    return columns_int, columns_object

In [ ]:
columns_int_tr, columns_object_tr = int_or_object(train_db)
columns_int_test, columns_object_test = int_or_object(test_db)

In [ ]:
# просмотр распределения значений в тренировочном датасете
for i in columns_int_tr:
   train_db.boxplot(i,figsize=(5, 5))
   plt.show() #для устранения вывода массива с названиями гистрограмм

In [ ]:
# вырежем вбрсоовые данные
train_db = train_db.loc[(train_db['year'] >= 2001) & (train_db['sellingprice'] <= 32500) & (train_db['odometer'] <= 180000)]

In [ ]:
# просмотр результа изменения датасета
train_db.info()

In [ ]:
test_db.info()

In [ ]:
columns_object_tr.remove('saledate')
columns_object_test.remove('saledate')

In [ ]:
for i in columns_object_tr:
    train_db[i] = train_db[i].astype('str').str.lower()

for i in columns_object_tr:
    train_db[i] = train_db[i].astype('str').str.rstrip()

for i in columns_object_test:
    test_db[i] = test_db[i].astype('str').str.lower()

for i in columns_object_test:
    test_db[i] = test_db[i].astype('str').str.rstrip()

In [ ]:
# поиск неявных дубликатов
for x in columns_object_tr:
    print('Уникальные значения для столбца', x, train_db[x].sort_values().unique(), '\n')

In [ ]:
# %%time
# for i in train_db['model'].unique():
#     x = i.split(' ')
#     if len(x) > 1:
#         pass
#     if len(x) > 2:
#         row = x[0] + ' ' + x[-1]
#         train_db.loc[train_db['model'] == i, 'model'] = row

In [ ]:
# %%time
# for i in test_db['model'].unique():
#     x = i.split(' ')
#     if len(x) > 1:
#         pass
#     if len(x) > 2:
#         row = x[0] + ' ' + x[-1]
#         test_db.loc[test_db['model'] == i, 'model'] = row

In [ ]:
# изменения неявных дубликатов
train_db['color'] = train_db['color'].str.replace('—', 'Nan')
train_db['interior'] = train_db['interior'].str.replace('—', 'Nan')
train_db['trim'] = train_db['trim'].str.replace('!', 'Nan')
train_db['trim'] = train_db['trim'].str.replace('+', 'Nan')

train_db['model'] = train_db['model'].str.replace('1','1 Series')
train_db['model'] = train_db['model'].str.replace('3','3 Series')
train_db['model'] = train_db['model'].str.replace('300','300-Class')

train_db['model'] = train_db['model'].str.replace('6','6 Series')

train_db['make'] = train_db['make'].str.replace('6','6 Series')
train_db['make'] = train_db['make'].str.replace('gmc truck','gmc')
train_db['make'] = train_db['make'].str.replace('mazda tk','mazda')
train_db['make'] = train_db['make'].str.replace('mercedes-b', 'mercedes-benz')
train_db['make'] = train_db['make'].str.replace('mercedes', 'mercedes-benz')
train_db['make'] = train_db['make'].str.replace('landrover','land rover')
train_db['make'] = train_db['make'].str.replace('vw', 'volkswagen')
train_db['make'] = train_db['make'].str.replace('Mercedes-Benzenz', 'mercedes-benz')
train_db['make'] = train_db['make'].str.replace('chev truck', 'chevrolet')
train_db['make'] = train_db['make'].str.replace('ford tk', 'ford')
train_db['make'] = train_db['make'].str.replace('hyundai tk', 'hyundai' )
train_db['make'] = train_db['make'].str.replace('mercedes-benz-benzenz-benzenz', 'mercedes-benz' )
train_db['make'] = train_db['make'].str.replace('mercedes-benz-benzenz', 'mercedes-benz' )
train_db['make'] = train_db['make'].str.replace('mercedes-benz-benz', 'mercedes-benz' )

In [ ]:
# поиск неявных дубликатов
for x in columns_object_test:
    print('Уникальные значения для столбца', x, test_db[x].sort_values().unique(), '\n')

In [ ]:
# изменения неявных дубликатов
test_db['color'] = test_db['color'].str.replace('—', 'Nan')
test_db['interior'] = test_db['interior'].str.replace('—', 'Nan')
test_db['trim'] = test_db['trim'].str.replace('!', 'Nan')
test_db['trim'] = test_db['trim'].str.replace('+', 'Nan')

test_db['model'] = test_db['model'].str.replace('1','1 Series')
test_db['model'] = test_db['model'].str.replace('3','3 Series')
test_db['model'] = test_db['model'].str.replace('300','300-Class')

test_db['model'] = test_db['model'].str.replace('6','6 Series')

test_db['make'] = test_db['make'].str.replace('6','6 Series')
test_db['make'] = test_db['make'].str.replace('gmc truck','gmc')
test_db['make'] = test_db['make'].str.replace('mazda tk','mazda')
test_db['make'] = test_db['make'].str.replace('mercedes-b', 'mercedes-benz')
test_db['make'] = test_db['make'].str.replace('mercedes', 'mercedes-benz')
test_db['make'] = test_db['make'].str.replace('landrover','land rover')
test_db['make'] = test_db['make'].str.replace('vw', 'volkswagen')
test_db['make'] = test_db['make'].str.replace('Mercedes-Benzenz', 'mercedes-benz')
test_db['make'] = test_db['make'].str.replace('chev truck', 'chevrolet')
test_db['make'] = test_db['make'].str.replace('ford tk', 'ford')
test_db['make'] = test_db['make'].str.replace('hyundai tk', 'hyundai' )
test_db['make'] = test_db['make'].str.replace('mercedes-benz-benzenz-benzenz', 'mercedes-benz' )
test_db['make'] = test_db['make'].str.replace('mercedes-benz-benzenz', 'mercedes-benz' )
test_db['make'] = test_db['make'].str.replace('mercedes-benz-benz', 'mercedes-benz' )

In [ ]:
# исследования наличия дубликатов в датасете
train_db.duplicated().sum()

In [ ]:
def overwriting_saledate(data):
    data['date'] = pd.to_datetime(data['saledate'], infer_datetime_format=True, utc=True)
    data['date'] = data['date'].dt.date



In [ ]:
train_db['saledate'] = overwriting_saledate(train_db)
test_db['saledate'] = overwriting_saledate(test_db)

In [ ]:
train_db = train_db.drop(columns=['saledate'], axis=1)
train_db = train_db.rename(columns={"date": "saledate"})

test_db = test_db.drop(columns=['saledate'], axis=1)
test_db = test_db.rename(columns={"date": "saledate"})

In [ ]:
train_db['saledate'] = pd.to_datetime(train_db['saledate'], format = '%Y-%m-%d')
test_db['saledate'] = pd.to_datetime(test_db['saledate'], format = '%Y-%m-%d')

In [ ]:
train_db.head()

In [ ]:
train_db.info()

In [ ]:
test_db.head()

In [ ]:
test_db.info()

**Вывод:**

###3. Изучение данных

In [ ]:
train_db['age_car'] = train_db['saledate'].dt.year - train_db['year']
test_db['age_car'] = test_db['saledate'].dt.year - test_db['year']

In [ ]:
train_db.head(2)

In [ ]:
# интенсивность использования
train_db['Usage_Intensity'] = train_db['odometer'] / train_db['age_car']
test_db['Usage_Intensity'] = test_db['odometer'] / test_db['age_car']

In [ ]:
# тут возможна ошибка
train_db.loc[train_db['Usage_Intensity'] <= 0, 'Usage_Intensity'] = 0
test_db.loc[test_db['Usage_Intensity'] <= 0, 'Usage_Intensity'] = 0

train_db['Usage_Intensity'] = train_db['Usage_Intensity'].replace([np.inf , -np.inf ], 0)
test_db['Usage_Intensity'] = train_db['Usage_Intensity'].replace([np.inf , -np.inf ], 0)

In [ ]:
test_db['Usage_Intensity'] = test_db['Usage_Intensity'].fillna(0)

In [ ]:
# Получим списки автомобилей по происхождению
american_cars = ['chevrolet', 'chrysler', 'gmc', 'cadillac',
                 'ford', 'buick', 'jeep', 'lincoln', 'ram',
                 'pontiac', 'dodge', 'hummer', 'plymouth', 'mercury',  'oldsmobile', 'plymouth','pontiac', 'saturn',
                'gmc truck','geo', 'chev truck','tesla', 'fisker', 'ford tk']


european_cars = ['mercedes-benz', 'mercedes', 'audi', 'bmw', 'lamborghini', 'porsche', 'volkswagen',
                 'volvo', 'fiat', 'mini', 'Maserati', 'alfa romeo', 'ferrari', 'saab', 'smart', 'maserati',
                'landrover', 'vw', 'mercedes-b']



british_cars = ['land rover', 'mclaren', 'jaguar', 'aston martin', 'bentley', 'lotus', 'rolls-royce']



asian_cars = ['hyundai', 'lexus',  'infiniti', 'mazda', 'toyota', 'genesis', 'kia',
              'acura', 'nissan', 'honda', 'mitsubishi', 'scion', 'suzuki','hyundai tk',  'daewoo',  'subaru', 'scion', 'isuzu','mazda tk']
base_cars = ['base1', 'base']

origin_dict = {}
for brand in american_cars:
    origin_dict[brand] = 'American'

for brand in european_cars:
    origin_dict[brand] = 'European'

for brand in british_cars:
    origin_dict[brand] = 'British'

for brand in asian_cars:
    origin_dict[brand] = 'Asian'

for brand in base_cars:
    origin_dict[brand] = 'Base'

train_db['Origin'] = np.vectorize(lambda x: origin_dict[x])(train_db['make'])
test_db['Origin'] = np.vectorize(lambda x: origin_dict[x])(test_db['make'])

In [ ]:
train_db.head(2)

In [ ]:
train_db['day'] = train_db['saledate'].dt.day
train_db['month'] = train_db['saledate'].dt.month
train_db['year_sale'] = train_db['saledate'].dt.year

In [ ]:
test_db['day'] = test_db['saledate'].dt.day
test_db['month'] = test_db['saledate'].dt.month
test_db['year_sale'] = test_db['saledate'].dt.year

In [ ]:
train_db.head(2)

In [ ]:
train_db = train_db.drop('saledate', axis = 1)
test_db = test_db.drop('saledate', axis = 1)

In [ ]:
train_db.head(2)

In [ ]:
train_db['lost_condition_on_1000'] = ((5 - train_db['condition']) / train_db['odometer'])  * 1000
test_db['lost_condition_on_1000'] = ((5 - test_db['condition']) / test_db['odometer'])  * 1000


In [ ]:
train_db.head(2)

In [ ]:
# condition / odometer  за каждый километр
# condition / odometer  * 1000 за каждую 1000 километров

# Изучение лучших моделей обучения

In [ ]:
def save_db(data, name):
    path = str(name) + '.csv'
    filepath = Path(path)
    filepath.parent.mkdir(parents=True, exist_ok=True)
    data.to_csv(filepath, index=False)

In [ ]:
# save_db(train_db, 'train_db')
# save_db(test_db, 'test_db')

In [ ]:
train_db = pd.read_csv('train_db.csv')
test_db = pd.read_csv('test_db.csv')

In [ ]:
# target_full
main_target_vin = pd.DataFrame()
main_target_vin = test_db['vin']
main_target_vin.shape

(110058,)

In [ ]:
train_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398187 entries, 0 to 398186
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              398187 non-null  int64  
 1   year                    398187 non-null  int64  
 2   make                    398187 non-null  object 
 3   model                   398187 non-null  object 
 4   trim                    398187 non-null  object 
 5   body                    398187 non-null  object 
 6   transmission            398187 non-null  object 
 7   vin                     398187 non-null  object 
 8   state                   398187 non-null  object 
 9   condition               398187 non-null  float64
 10  odometer                398187 non-null  float64
 11  color                   398187 non-null  object 
 12  interior                398187 non-null  object 
 13  seller                  398187 non-null  object 
 14  sellingprice        

In [ ]:
test_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110058 entries, 0 to 110057
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              110058 non-null  int64  
 1   year                    110058 non-null  int64  
 2   make                    110058 non-null  object 
 3   model                   110058 non-null  object 
 4   trim                    110058 non-null  object 
 5   body                    110058 non-null  object 
 6   transmission            110058 non-null  object 
 7   vin                     110058 non-null  object 
 8   state                   110058 non-null  object 
 9   condition               110058 non-null  float64
 10  odometer                110058 non-null  float64
 11  color                   110058 non-null  object 
 12  interior                110058 non-null  object 
 13  seller                  110058 non-null  object 
 14  age_car             

In [ ]:
test_db = test_db.drop('Unnamed: 0', axis = 1)
train_db = train_db.drop('Unnamed: 0', axis = 1)

In [ ]:
display(train_db.head(2))

,year,make,model,trim,body,transmission,vin,state,condition,odometer,...,interior,seller,sellingprice,age_car,Usage_Intensity,Origin,day,month,year_sale,lost_condition_on_1000
0,2011,ford,edge,sel,suv,automatic,2fmdk3jc4bba41556,md,4.2,111041.0,...,black,santander consumer,12500,4,27760.25,American,1,6,2015,0.007205
1,2014,ford,fusion,se,sedan,automatic,3fa6p0h75er208976,mo,3.5,31034.0,...,black,ars/avis budget group,14500,1,31034.00,American,24,2,2015,0.048334


In [ ]:
def split_data_to_ml(data, to_drop, target):
    '''функция делит датасеты на параметры и таргет'''
    features = data.drop(to_drop, axis = 1)
    target = data[target]

    return features, target


In [ ]:
train_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398187 entries, 0 to 398186
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   year                    398187 non-null  int64  
 1   make                    398187 non-null  object 
 2   model                   398187 non-null  object 
 3   trim                    398187 non-null  object 
 4   body                    398187 non-null  object 
 5   transmission            398187 non-null  object 
 6   vin                     398187 non-null  object 
 7   state                   398187 non-null  object 
 8   condition               398187 non-null  float64
 9   odometer                398187 non-null  float64
 10  color                   398187 non-null  object 
 11  interior                398187 non-null  object 
 12  seller                  398187 non-null  object 
 13  sellingprice            398187 non-null  int64  
 14  age_car             

In [ ]:
# НУЖЕН МОЩНЫЙ КОМП ЧТО БЫ ЗАКОДИРОВАТЬ ВСЕ КАТЕГОРИАЛЬНЫЕ ПРИЗНАКИ
# деление датасета на признаки и таргет
features, target = split_data_to_ml(
    train_db, ['sellingprice', 'vin','make', 'model', 'trim','body', 'state', 'color', 'interior', 'seller'], 'sellingprice')

test_db = test_db.drop(['vin','make', 'model', 'body', 'trim','state', 'color', 'interior', 'seller'], axis = 1)

In [ ]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398187 entries, 0 to 398186
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   year                    398187 non-null  int64  
 1   transmission            398187 non-null  object 
 2   condition               398187 non-null  float64
 3   odometer                398187 non-null  float64
 4   age_car                 398187 non-null  int64  
 5   Usage_Intensity         398187 non-null  float64
 6   Origin                  398187 non-null  object 
 7   day                     398187 non-null  int64  
 8   month                   398187 non-null  int64  
 9   year_sale               398187 non-null  int64  
 10  lost_condition_on_1000  398187 non-null  float64
dtypes: float64(4), int64(5), object(2)
memory usage: 33.4+ MB


In [ ]:
test_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110058 entries, 0 to 110057
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   year                    110058 non-null  int64  
 1   transmission            110058 non-null  object 
 2   condition               110058 non-null  float64
 3   odometer                110058 non-null  float64
 4   age_car                 110058 non-null  int64  
 5   Usage_Intensity         110058 non-null  float64
 6   Origin                  110058 non-null  object 
 7   day                     110058 non-null  int64  
 8   month                   110058 non-null  int64  
 9   year_sale               110058 non-null  int64  
 10  lost_condition_on_1000  110058 non-null  float64
dtypes: float64(4), int64(5), object(2)
memory usage: 9.2+ MB


In [ ]:
# деление датасета train на тренировочную и вилдационную выборку
features_train, features_valid, target_train, target_valid = train_test_split(
   features, target, test_size=0.30, random_state = 12345, shuffle = True)


In [ ]:
# features_train = features_train.drop('Unnamed: 0', axis = 1)
# features_valid = features_valid.drop('Unnamed: 0', axis = 1)


In [ ]:
#категориальные признаки
ohe_features = features_train.select_dtypes(include='object').columns.to_list()
ohe_features

['transmission', 'Origin']

In [ ]:
#численные признаки
num_features = features_train.select_dtypes(exclude='object').columns.to_list()

num_features

['year',
 'condition',
 'odometer',
 'age_car',
 'Usage_Intensity',
 'day',
 'month',
 'year_sale',
 'lost_condition_on_1000']

In [ ]:
%%time
# кодирование данных техникой OHE
encoder_ohe = OneHotEncoder(drop='first', handle_unknown='error', sparse=False)

# обучаем энкодер
encoder_ohe.fit(features_train[ohe_features])

# добавляем закодированные признаки датасет

features_train[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_train[ohe_features])

# удаляем незакодированные категориальные признаки
features_train = features_train.drop(ohe_features, axis=1)

#скелер
scaler = StandardScaler()

# обучаем его
features_train[num_features] = scaler.fit_transform(features_train[num_features])

features_train.head()
features_train.shape

CPU times: total: 734 ms
Wall time: 750 ms


(278730, 14)

In [ ]:
%%time
# энкодером, который обучен на ТРЕНИРОВОЧНОЙ ВЫБОРКЕ, кодируем тестовую

features_valid[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_valid[ohe_features])

features_valid = features_valid.drop(ohe_features, axis=1)

# скелером масштабируем тестовую
features_valid[num_features] = scaler.transform(
    features_valid[num_features]
)

# смотрим на результат
features_valid.head()
features_valid.shape

CPU times: total: 312 ms
Wall time: 311 ms


(119457, 14)

In [ ]:
%%time

test_db[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(test_db[ohe_features])

# удаляем незакодированные категориальные признаки
test_db = test_db.drop(ohe_features, axis=1)


# обучаем его
test_db[num_features] = scaler.transform(test_db[num_features])

test_db.head()
test_db.shape

CPU times: total: 266 ms
Wall time: 270 ms


(110058, 14)

План реализации ML
Пробую модели:
- RF regression
- Регрессия LASSO
- Гребневая регрессия (ридж-регрессия)
- Регрессия ElasticNet
- GradientBoostingRegressor
- catboost


https://habr.com/ru/companies/vk/articles/513842/

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
%%time
model = RandomForestRegressor(random_state = 12345, n_estimators = 40, max_depth = 10) #  инициализируйте модель с наилучшим значением метрики RMSE
model.fit(features_train, target_train) # обучите модель на тренировочной выборке
predictions_valid = model.predict(features_valid) # получите предсказания модели на валидационной выборке

result = mean_absolute_percentage_error(target_valid, predictions_valid) # посчитайте значение метрики RMSE на валидационной выборке
print("RMSE наилучшей модели на валидационной выборке:", result)

RMSE наилучшей модели на валидационной выборке: 0.41276489360638474
CPU times: total: 46.7 s
Wall time: 48.6 s


In [ ]:
from sklearn.linear_model import LassoCV

In [ ]:
%%time
model = LassoCV(random_state = 12345)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)

result = mean_absolute_percentage_error(target_valid, predictions_valid) # посчитайте значение метрики RMSE на валидационной выборке
print("RMSE наилучшей модели на валидационной выборке:", result)

RMSE наилучшей модели на валидационной выборке: 0.4488781863658132
CPU times: total: 4.31 s
Wall time: 2.47 s


In [ ]:
from sklearn.linear_model import RidgeCV


In [ ]:
%%time
model = RidgeCV()
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)

result = mean_absolute_percentage_error(target_valid, predictions_valid) # посчитайте значение метрики RMSE на валидационной выборке
print("RMSE наилучшей модели на валидационной выборке:", result)

RMSE наилучшей модели на валидационной выборке: 0.44918798509328395
CPU times: total: 375 ms
Wall time: 230 ms


In [ ]:
%%time
model = Ridge()
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)

result = mean_absolute_percentage_error(target_valid, predictions_valid) # посчитайте значение метрики RMSE на валидационной выборке
print("RMSE наилучшей модели на валидационной выборке:", result)

RMSE наилучшей модели на валидационной выборке: 0.44918799262534614
CPU times: total: 109 ms
Wall time: 70.5 ms


In [ ]:
from sklearn.linear_model import ElasticNetCV

In [ ]:
%%time
model = ElasticNetCV(random_state = 12345)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)

result = mean_absolute_percentage_error(target_valid, predictions_valid) # посчитайте значение метрики RMSE на валидационной выборке
print("RMSE наилучшей модели на валидационной выборке:", result)


RMSE наилучшей модели на валидационной выборке: 0.8340861424833989
CPU times: total: 1.55 s
Wall time: 1.42 s


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
%%time
model = GradientBoostingRegressor(random_state = 12345)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)

result = mean_absolute_percentage_error(target_valid, predictions_valid) # посчитайте значение метрики RMSE на валидационной выборке
print("RMSE наилучшей модели на валидационной выборке:", result)

RMSE наилучшей модели на валидационной выборке: 0.41458929270232164
CPU times: total: 55.4 s
Wall time: 57.1 s


In [ ]:
from catboost import CatBoostRegressor
# https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb

In [ ]:
cb_learn_rate = 0.006
n_iterations = 80000
early_stop_rounds = 400

opt_catboost_params = {'iterations' : n_iterations,
                       'learning_rate' : cb_learn_rate,
                       'depth': 7,
                       'bootstrap_type' : 'Bernoulli',
                       'random_strength': 1,
                       'min_data_in_leaf': 10,
                       'l2_leaf_reg': 3,
                       'loss_function' : 'MAPE',
                       'eval_metric' : 'MAPE',
                       'grow_policy' : 'Depthwise',
                       'max_bin' : 1024,
                       'model_size_reg' : 0,
                       'task_type' : 'GPU',
                       'od_type' : 'IncToDec',
                       'od_wait' : 100,
                       'metric_period' : 500,
                       'verbose' : 500,
                       'subsample' : 0.8,
                       'od_pval' : 1e-10,
                       'max_ctr_complexity' : 8,
                       'has_time': False,
                       'simple_ctr' : 'FeatureFreq',
                       'combinations_ctr': 'FeatureFreq',
                       'random_seed' : 13}

In [ ]:
%%time
cb_reg_1 = CatBoostRegressor(**opt_catboost_params)
cb_reg_1.fit(features_train, target_train)
cb_preds = cb_reg_1.predict(features_valid)

# calculate mean squared error on val sub-set preds
result = mean_absolute_percentage_error(cb_preds, predictions_valid)
print(result)


0:	learn: 0.7575753	total: 50.6ms	remaining: 1h 7m 25s
500:	learn: 0.7292708	total: 6.33s	remaining: 16m 45s
1000:	learn: 0.7287499	total: 12.6s	remaining: 16m 37s
1500:	learn: 0.7287239	total: 19s	remaining: 16m 31s
2000:	learn: 0.7287227	total: 25.2s	remaining: 16m 22s
2500:	learn: 0.7287226	total: 31.6s	remaining: 16m 18s
3000:	learn: 0.7287226	total: 37.8s	remaining: 16m 11s
3500:	learn: 0.7287227	total: 44.2s	remaining: 16m 4s
4000:	learn: 0.7287226	total: 50.5s	remaining: 15m 59s
4500:	learn: 0.7287226	total: 56.8s	remaining: 15m 52s
5000:	learn: 0.7287226	total: 1m 3s	remaining: 15m 47s
5500:	learn: 0.7287226	total: 1m 9s	remaining: 15m 40s
6000:	learn: 0.7287227	total: 1m 15s	remaining: 15m 35s
6500:	learn: 0.7287227	total: 1m 22s	remaining: 15m 29s
7000:	learn: 0.7287227	total: 1m 28s	remaining: 15m 22s
7500:	learn: 0.7287226	total: 1m 34s	remaining: 15m 16s
8000:	learn: 0.7287226	total: 1m 41s	remaining: 15m 9s
8500:	learn: 0.7287227	total: 1m 47s	remaining: 15m 2s
9000:	lear

73000:	learn: 0.7287226	total: 15m 36s	remaining: 1m 29s
73500:	learn: 0.7287226	total: 15m 42s	remaining: 1m 23s
74000:	learn: 0.7287226	total: 15m 49s	remaining: 1m 16s
74500:	learn: 0.7287226	total: 15m 55s	remaining: 1m 10s
75000:	learn: 0.7287227	total: 16m 1s	remaining: 1m 4s
75500:	learn: 0.7287226	total: 16m 7s	remaining: 57.7s
76000:	learn: 0.7287226	total: 16m 14s	remaining: 51.3s
76500:	learn: 0.7287226	total: 16m 20s	remaining: 44.9s
77000:	learn: 0.7287226	total: 16m 27s	remaining: 38.4s
77500:	learn: 0.7287227	total: 16m 33s	remaining: 32s
78000:	learn: 0.7287226	total: 16m 39s	remaining: 25.6s
78500:	learn: 0.7287226	total: 16m 46s	remaining: 19.2s
79000:	learn: 0.7287227	total: 16m 52s	remaining: 12.8s
79500:	learn: 0.7287227	total: 16m 58s	remaining: 6.39s
79999:	learn: 0.7287226	total: 17m 5s	remaining: 0us
1.9373800548098965
CPU times: total: 35min 37s
Wall time: 18min 30s


In [ ]:
predict = cb_reg_1.predict(test_db)

In [ ]:
df = pd.DataFrame(data = {'vin': main_target_vin, 'sellingprice': predict}, index = range(len(main_target_vin)))

In [ ]:
save_db(df, 'sample_submission_1')